In [ ]:
from basic import cursor, get, count, multicount

以下引入数据，`family_numbers`为各年总家庭数，`rural_numbers`为各年样本中的农村总数，

In [ ]:
family_numbers = multicount('famecon')

rural_numbers = multicount(
    'famecon',
    conditions="urban{year}=0",
    condition_format=['', '12', '14', '16', '18']
    )

rural_agri_numbers = multicount(
    'famecon',
    conditions=[
        'urban=0 and fk1=1',
        'urban12=0 and fk1l=1',
        'urban14=0 and fk1l=1',
        'urban16=0 and fk1l=1',
        'urban18=0 and fk1l=1'
    ]
)

计算农村从事农业的比例：

In [ ]:
rural_agri_ratios = [b/a for a, b in zip(rural_numbers, rural_agri_numbers)]